<a href="https://colab.research.google.com/github/royal-dit/tensorflow-deeplearning/blob/main/food_vision_project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-a6c2f197-6424-5bf5-2d92-be34497e1e66)


In [24]:
!wget https://raw.githubusercontent.com/royal-dit/tensorflow-deeplearning/main/helper_func.py 

--2022-10-01 05:35:33--  https://raw.githubusercontent.com/royal-dit/tensorflow-deeplearning/main/helper_func.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12051 (12K) [text/plain]
Saving to: ‘helper_func.py.1’

helper_func.py.1    100%[===================>]  11.77K  --.-KB/s    in 0s      

2022-10-01 05:35:34 (79.0 MB/s) - ‘helper_func.py.1’ saved [12051/12051]



In [25]:
from helper_func import  unzip_data,create_tensorboard_callback,plot_loss_curves,make_confusion_matrix,compare_historys,walk_through_dir

In [26]:
!wget https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/07_food_vision_milestone_project_1.ipynb

--2022-10-01 05:35:42--  https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/07_food_vision_milestone_project_1.ipynb
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘07_food_vision_milestone_project_1.ipynb’

07_food_vision_mile     [ <=>                ] 146.09K  --.-KB/s    in 0.01s   

2022-10-01 05:35:42 (9.92 MB/s) - ‘07_food_vision_milestone_project_1.ipynb’ saved [149593]



In [ ]:
import tensorflow_datasets as tfds

In [ ]:
datasets_list = tfds.list_builders() #get all avilable datasets in tfd
print("food101" in datasets_list)

True


In [ ]:
# load in the data (takeks 5-6 min in google colab)
(train_data,test_data),ds_info = tfds.load(name = "food101",
                                           split = ["train","validation"],
                                           shuffle_files=True,
                                           as_supervised=True, #gets data returned in tuple format
                                           with_info = True
                                           )

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/75750 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/food101/2.0.0.incomplete7O9LER/food101-train.tfrecord*...:   0%|          | 0/…

Generating validation examples...:   0%|          | 0/25250 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/food101/2.0.0.incomplete7O9LER/food101-validation.tfrecord*...:   0%|         …

Dataset food101 downloaded and prepared to ~/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.


In [27]:
class_names = ds_info.features["label"].names
class_names[:10]

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito']

In [28]:
train_one_sample = train_data.take(1)

In [29]:
train_one_sample

<TakeDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [30]:
for image ,label in train_one_sample:
  image.shape,label

In [42]:
#make a function for processing image
import tensorflow as tf
def preprocesses_img(image,label,img_shape =224):
  """
  converts image datatype from uint8 -> float32
  """
  image = tf.image.resize(image,[img_shape,img_shape])
  # image = image/255. not reuired with effiecient netb0
  return tf.cast(image,tf.float32),label  

In [43]:
preprocess_img = preprocesses_img(image,label)[0]

**batch and prepare dataset**

In [44]:
train_data =  train_data.map(map_func = preprocesses_img,num_parallel_calls = tf.data.AUTOTUNE)
train_data = train_data.shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)
#map processing to test data
test_data = test_data.map(preprocesses_img,num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)



In [45]:
train_data,test_data

(<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

In [46]:
#create modelling callbacks
checkpoint_path = "model_checkpoint/cp.ckpt"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only = True,
                                                         monitor="val_accuracy",
                                                         save_best_only = True,
                                                         verbose = 0
                                                         )




In [49]:
#turn on mixed precision
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")


In [58]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
input_shape = (224,224,3)
base_model = tf.keras.applications.EfficientNetB0(include_top = False)
base_model.trainable = False
inputs = layers.Input(shape = input_shape,name = "input_layer")
x =  base_model(inputs,training = False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax",dtype = tf.float32,name = "softmax_float32")(x)
model = tf.keras.Model(inputs,outputs)

model.compile (
    loss = "sparse_categorical_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]

)


In [63]:
history_101_food_classes = model.fit (train_data,
                                      epochs = 5,
                                      steps_per_epoch = (len(train_data)),
                                      validation_data = test_data,
                                      validation_steps = int(0.15*len(test_data)))
                                  

  

Epoch 1/5
2368/2368 [==============================] - 203s 78ms/step - loss: 1.8191 - accuracy: 0.5586 - val_loss: 1.2151 - val_accuracy: 0.6780
Epoch 2/5
2368/2368 [==============================] - 184s 77ms/step - loss: 1.2944 - accuracy: 0.6662 - val_loss: 1.1271 - val_accuracy: 0.7002
Epoch 3/5
2368/2368 [==============================] - 183s 77ms/step - loss: 1.1427 - accuracy: 0.7026 - val_loss: 1.0841 - val_accuracy: 0.7060
Epoch 4/5
2368/2368 [==============================] - 177s 74ms/step - loss: 1.0456 - accuracy: 0.7258 - val_loss: 1.0704 - val_accuracy: 0.7113
Epoch 5/5
2368/2368 [==============================] - 178s 75ms/step - loss: 0.9734 - accuracy: 0.7433 - val_loss: 1.0537 - val_accuracy: 0.7124
